In [1]:
import tqdm
import gzip
import numpy as np
from models.XStream_River import xStream
from sklearn.metrics import average_precision_score, roc_auc_score

In [2]:
# data = gzip.open("data/Static/cancer.gz", "r")
# data = gzip.open("data/Static/ionosphere.gz", "r")
# data = gzip.open("data/Static/telescope.gz", "r")
data = gzip.open("data/Static/indians.gz", "r")
# data = gzip.open("data/Static/gisette.gz", "r")
# data = gzip.open("data/Static/isolet.gz", "r")
# data = gzip.open("data/Static/letter.gz", "r")
# data = gzip.open("data/Static/madelon.gz", "r")

In [3]:
X, y = [], []

for i in data:
  i = (i.decode('utf-8')).split(",")
  i = [float(x) for x in i]
  X.append(np.array(i[:-1]))
  y.append(i[-1])

In [4]:
window_size = int(0.25*len(y))
k = 100 
n_chains = 100 
depth = 15

cf = xStream(num_components=k, n_chains=n_chains, depth=depth, window_size=window_size) 

all_scores = []

for i, x in enumerate(tqdm.tqdm(X)):
  cf.learn_one(x)
  if i>=window_size:
    anomalyscore = -cf.predict_one(x)
    all_scores.append(anomalyscore[0])

100%|██████████| 538/538 [00:21<00:00, 25.25it/s]


In [5]:
y_adjusted = y[window_size:window_size+len(all_scores)]

In [6]:
# Computation for Mean Average Precision

chunks = [all_scores[x:x+window_size] for x in range(0, len(all_scores), window_size)]
y_chunks = [y_adjusted[x:x+window_size] for x in range(0, len(y_adjusted), window_size)]

AP_window = []

for i in range(len(y_chunks)-1):
  score = average_precision_score(y_chunks[i], chunks[i])
  AP_window.append(score)


c:\Users\aramosvela\Documents\Data_Science\uni_env\lib\site-packages\sklearn\metrics\_ranking.py:891: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
c:\Users\aramosvela\Documents\Data_Science\uni_env\lib\site-packages\sklearn\metrics\_ranking.py:891: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


In [7]:
OAP = average_precision_score(y_adjusted, all_scores) 
MAP = sum(AP_window)/len(AP_window)
AUC = roc_auc_score(y_adjusted, all_scores)

print("XStream: OAP =", OAP,"\n\t ",
      "MAP =", MAP, "\n\t ", 
      "AUC =", AUC)

XStream: OAP = 0.09013973000131735 
	  MAP = 0.18008014577104645 
	  AUC = 0.49928098935864257
